In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Activation,Flatten

from tensorflow import keras
from tensorflow.keras import layers

from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.keras.__version__)
print(tf.config.list_physical_devices('GPU'))

In [ ]:
data = pd.read_csv('AUDCAD/AUD_CAD_rev.csv')


In [ ]:
# GRYP event embedding
GRYP_IDX = {}
for idx, value in enumerate( list(data.event.unique())):
    GRYP_IDX[value] = idx

data['event'].replace(GRYP_IDX, inplace= True)


In [ ]:
# string to index and change -1 to 2
REPLACE_INDEX = {'buy':0, 'hold':1, 'sell':2, -1:2}
data.replace(REPLACE_INDEX, inplace= True)
# one hot label index
BUY_HOLD_SELL_INDEX = {'buy': [0,1,0,0], 'hold': [0,0,1,0], 'sell': [0,0,0,1]}

In [ ]:
# remove all nan
print(f'before: {data.shape}')
data.dropna(subset=['min_40', 'ratio top'],inplace=True)
data = data.reset_index(drop=True)
# train val test split 
Xtrain = data[(data['datetime']>='2018-06-25T09:00:00.000000000Z') & ((data['datetime']<='2021-09-13T00:45:00.000000000Z'))]
Xtrain.reset_index(drop=True, inplace=True)
Xvalid = data[(data['datetime']>='2021-09-13T01:00:00.000000000Z') & ((data['datetime']<='2022-02-04T07:45:00.000000000Z'))]
Xvalid.reset_index(drop=True, inplace=True)
Xtest = data[(data['datetime']>='2022-02-04T08:00:00.000000000Z') & ((data['datetime']<='2022-06-30T13:30:00.000000000Z'))]
Xtest.reset_index(drop=True, inplace=True)

print(f'after remove null values: {data.shape} \ntrain size: {train_size}\nvalidtion size: {valid_size}\ntest size: {test_size}')


In [ ]:
def generate_sequence(data, window_size):
    train_dt_ori, train_dt_scaled, target_minprice, target_maxprice, target_minp_scaled,\
    target_maxp_scaled, price_scaler_max, price_scaler_min = [], [], [], [], [], [], [], []

    train_dt_bb1,train_dt_bb2,train_dt_bb3,train_dt_bb4,train_dt_bb5,train_dt_bb6,train_dt_bb7,train_dt_bb8 = [], [], [], [], [], [], [], []
    train_dt_trend,train_dt_change,train_dt_break=[],[],[]
    train_dt_rsi15,train_dt_rsi25,train_dt_rsi35 = [], [], []
    train_dt_wr_atr=[]
    target_bhs = []

    seqlen = window_size # change window size according to label！

    scaler = MinMaxScaler()
    for index, row in data.iterrows(): 
        if index <= len(data)-seqlen:
            # OHLC numerical
            train_dt_ori.append(data.loc[index:seqlen-1+index, ['open', 'high', 'low', 'close']].values)
            # Categorical
            train_dt_bb1.append(data.loc[index:seqlen-1+index, ['bb_event1']].values)
            train_dt_bb2.append(data.loc[index:seqlen-1+index, ['bb_event2']].values)
            train_dt_bb3.append(data.loc[index:seqlen-1+index, ['bb_event3']].values)
            train_dt_bb4.append(data.loc[index:seqlen-1+index, ['bb_event4']].values)
            train_dt_bb5.append(data.loc[index:seqlen-1+index, ['bb_event5']].values)
            train_dt_bb6.append(data.loc[index:seqlen-1+index, ['bb_event6']].values)
            train_dt_bb7.append(data.loc[index:seqlen-1+index, ['bb_event7']].values)
            train_dt_bb8.append(data.loc[index:seqlen-1+index, ['bb_event8']].values)
            train_dt_trend.append(data.loc[index:seqlen-1+index, ['Trend']].values)
            train_dt_change.append(data.loc[index:seqlen-1+index, ['Change']].values)
            train_dt_break.append(data.loc[index:seqlen-1+index, ['Break']].values)
            train_dt_rsi15.append(data.loc[index:seqlen-1+index, ['rsi15']].values)
            train_dt_rsi25.append(data.loc[index:seqlen-1+index, ['rsi25']].values)
            train_dt_rsi35.append(data.loc[index:seqlen-1+index, ['rsi35']].values)
            
            # numerical scale
            train_dt_wr_atr.append(scaler.fit_transform(data.loc[index:seqlen-1+index, ['wr15', 'wr25', 'wr35', 'atr15', 'atr25','atr35']].values))
            # min max scale within windows size
            train_dt_scaled.append(scaler.fit_transform(data.loc[index:seqlen-1+index, ['open', 'high', 'low', 'close']].values)
                                   
            # every window size/seq length predict min_10 label
            tmp_minprice = data.loc[seqlen-1+index, f'min_{window_size}'].tolist()
            target_minprice.append(tmp_minprice)
            # every window size/seq length predict max_10 label
            tmp_maxprice = data.loc[seqlen-1+index, f'max_{window_size}'].tolist()
            target_maxprice.append(tmp_maxprice)

            # taken bhs label
            target_bhs.append( data.loc[seqlen-1+index, ['ws10_pt8_sl15']].values)

            # max/min price within ws seqlength
            max_v = max(data.loc[index:seqlen-1+index, ['open', 'high', 'low', 'close']].max())
            min_v = min(data.loc[index:seqlen-1+index, ['open', 'high', 'low', 'close']].min())
            # min_10 max_10 label scaled 
            target_minp_scaled.append((tmp_minprice-min_v)/(max_v-min_v))
            target_maxp_scaled.append((tmp_maxprice-min_v)/(max_v-min_v))
            # save
            price_scaler_max.append(max_v)
            price_scaler_min.append(min_v)
            
    # change to np.array astype()
    train_arr_ohlc_scaled = np.array(train_dt_scaled).astype('float32')
    train_arr_bb1 = np.array(train_dt_bb1).astype('int64')
    train_arr_bb2 = np.array(train_dt_bb2).astype('int64')
    train_arr_bb3 = np.array(train_dt_bb3).astype('int64')
    train_arr_bb4 = np.array(train_dt_bb4).astype('int64')
    train_arr_bb5 = np.array(train_dt_bb5).astype('int64')
    train_arr_bb6 = np.array(train_dt_bb6).astype('int64')
    train_arr_bb7 = np.array(train_dt_bb7).astype('int64')
    train_arr_bb8 = np.array(train_dt_bb8).astype('int64')
    train_arr_trend = np.array(train_dt_trend).astype('int64')
    train_arr_change = np.array(train_dt_change).astype('int64')
    train_arr_break = np.array(train_dt_break).astype('int64')
    train_arr_rsi15 = np.array(train_dt_rsi15).astype('int64')
    train_arr_rsi25 = np.array(train_dt_rsi25).astype('int64')
    train_arr_rsi35 = np.array(train_dt_rsi35).astype('int64')
    train_arr_wr_atr = np.array(train_dt_wr_atr).astype('float32')
    target_minpArr_scaled = np.array(target_minp_scaled).astype('float32')
    target_maxpArr_scaled = np.array(target_maxp_scaled).astype('float32') 
    target_arr_bhs = np.array(target_bhs).astype('int64')
    
    # save features and labels in dict {'OHLC' : np.array([ohlc])}
    train_x_dict = {'OHLC':train_arr_ohlc_scaled,'bb1': train_arr_bb1,'bb2': train_arr_bb2,'bb3': train_arr_bb3,
                    'bb4': train_arr_bb4,'bb5': train_arr_bb5,'bb6': train_arr_bb6,'bb7': train_arr_bb7,
                    'bb8': train_arr_bb8,'TREND': train_arr_change,'CHANGE': train_arr_change,'BREAK': train_arr_break,
                    'rsi15': train_arr_rsi15,'rsi25': train_arr_rsi25,'rsi35': train_arr_rsi35,'wr_atr':train_arr_wr_atr}

    train_y_dict = {'minp': target_minpArr_scaled, 'maxp': target_maxpArr_scaled, 'bhs':target_arr_bhs}
        
        
    return train_x_dict,train_y_dict, price_scaler_max,price_scaler_min,target_minprice, target_maxprice


In [ ]:
# train set
train_x_dict,train_y_dict,price_scaler_max,price_scaler_min,train_minprice, train_maxprice = generate_sequence(Xtrain,10)
# validation set
valid_x_dict,valid_y_dict,val_price_scaler_max,val_price_scaler_min,valid_minprice, valid_maxprice = generate_sequence(Xvalid,10)
# test set
test_x_dict,test_y_dict,test_price_scaler_max,test_price_scaler_min,test_minprice, test_maxprice = generate_sequence(Xtest,10)

# Evaluation

In [ ]:
# accuary calculation
def evaluate_regression_prediction(y_pred, y_true, model_name, printFlag=True):

    errors = np.array(y_pred) - np.array(y_true)
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()
    mape = np.abs(100*errors/y_true).mean()
    if printFlag:
        print(model_name + ':')
        print('Mean Absolute Error: {:.8f}'.format(mae))
        print('Mean Absolute Percentage Error: {:.8f}'.format(mape))
        print('Mean Square Error: {:.8f}'.format(mse))
        print('Root Mean Square Error: {:.8f}'.format(rmse))
        print('')

    return mae, mape, mse, rmse

def inverse_buyholdsell_label(output_ls):
    # softmax vector find max one
    temp = [0,0,0]
    idx = 0   
    for i in output_ls:
        if i == max(output_ls):
            temp[idx] = 1
        idx+=1
    return temp

# Predict test
def visualize_result(model, train_x_dict, train_y_dict,price_scaler_min, price_scaler_max,train_minprice,train_maxprice):
    y_pred = model.predict(train_x_dict)
    print('-' * 100)
    print('MinMax Price Prediction Result: ')

    # max
    y_pred[0]
    # min
    y_pred[1]
    # buy hold sell label
    # y_pred[2]

    # Inverse prediction to orginal scale
    min_output,max_output = [],[]
    for i in range(len(y_pred[0])):
        #print(price_scaler_max[i] - price_scaler_min[i], y_pred[0][i] , price_scaler_min[i] )
        max_output.append( (y_pred[0][i] * (price_scaler_max[i] - price_scaler_min[i])) + price_scaler_min[i] )
        min_output.append( (y_pred[1][i] * (price_scaler_max[i] - price_scaler_min[i])) + price_scaler_min[i] )

    a,b,c,d =evaluate_regression_prediction(max_output,train_maxprice,'Model')
    a,b,c,d =evaluate_regression_prediction(min_output,train_minprice,'Model')


    # Prediction result on Test set
    plt.figure(2)
    plt.figure(figsize=(15,6))
    plt.plot(max_output, label='Max Predict')
    #plt.plot(min_output, label='Min Predict')
    #plt.plot(train_minprice, label='Actual Min Price') 
    plt.plot(train_maxprice, label='Actual Max Price') 
    plt.title('Predict Max & Actual Max')
    plt.legend(loc = 'upper right')
    
    plt.figure(2)
    plt.figure(figsize=(15,6))
    plt.plot(min_output, label='Min Predict')
    plt.plot(train_minprice, label='Actual Min Price') 
    plt.title('Predict Min & Actual Min')
    plt.legend(loc = 'upper right')

    # Inverse buy hold sell prediction label
    if len(y_pred) >2:
        res = []
        for i in y_pred[2]:
            pre = inverse_buyholdsell_label(list(i))
            res.append(pre)
        

        # res
        ground_true = train_y_dict['bhs']
        ground_true_matrix = to_categorical(ground_true)
        # print(f'Accuracy for predicting buy hold sell label: {accuracy_score(res, valid_y_dict['bhs']):.4f}')
        # accuracy_score(res, ground_true_matrix)

        print(f'Accuracy for predicting buy hold sell label: {accuracy_score(ground_true_matrix,res):.4f}')
        print(classification_report(ground_true_matrix,res))
        print('-' * 100)
        
def visualize_history(history):
    # visual_loss = ['maxp_loss', 'minp_loss', 'bhs_loss','val_maxp_loss', 'val_minp_loss', 'val_bhs_loss']
    visual_loss = ['maxp_loss', 'minp_loss','val_maxp_loss', 'val_minp_loss']
    visual_acc = ['Train Accuracy', 'Validation Accuracy']

    plt.figure(2)
    plt.figure(figsize=(15,6))
    for i in visual_loss:
        history.history[i]
        plt.plot(history.epoch,history.history[i], label =i )
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(loc = 'upper right')

    plt.figure(2)
    history.history[i]
    plt.bar(x = visual_acc, height = [history.history['bhs_acc'][-1],history.history['val_bhs_acc'][-1]] )
    plt.xlabel('Accuracy')
    plt.ylabel('Train & Validation')
    plt.title('Label BuyHoldSell Prediction Accuracy')
    plt.show()

## LSTM model 

In [ ]:
def buildModel(seqlen, EMBED_DIM=4): 
    # feature input
    input_ohlc = layers.Input((seqlen,4), dtype=tf.float32, name='OHLC')
    input_bb1 = layers.Input((seqlen,), dtype=tf.int64, name='bb1')
    input_bb2 = layers.Input((seqlen,), dtype=tf.int64, name='bb2')
    input_bb3 = layers.Input((seqlen,), dtype=tf.int64, name='bb3')
    input_bb4 = layers.Input((seqlen,), dtype=tf.int64, name='bb4')
    input_bb5 = layers.Input((seqlen,), dtype=tf.int64, name='bb5')
    input_bb6 = layers.Input((seqlen,), dtype=tf.int64, name='bb6')
    input_bb7 = layers.Input((seqlen,), dtype=tf.int64, name='bb7')
    input_bb8 = layers.Input((seqlen,), dtype=tf.int64, name='bb8')
    input_TREND = layers.Input((seqlen,), dtype=tf.int64, name='TREND')
    input_CHANGE = layers.Input((seqlen,), dtype=tf.int64, name='CHANGE')
    input_BREAK = layers.Input((seqlen,), dtype=tf.int64, name='BREAK')
    input_rsi15 = layers.Input((seqlen,), dtype=tf.int64, name='rsi15')
    input_rsi25 = layers.Input((seqlen,), dtype=tf.int64, name='rsi25')
    input_rsi35 = layers.Input((seqlen,), dtype=tf.int64, name='rsi35')
    input_wr_atr = layers.Input((seqlen,6), dtype=tf.float32, name='wr_atr')
    # categorical embedding layer (input_dim = categorical number+1)
    emb_bb1 = layers.Embedding(3, EMBED_DIM, name='bb1_embedding')(input_bb1)
    emb_bb2 = layers.Embedding(3, EMBED_DIM, name='bb2_embedding')(input_bb2)
    emb_bb3 = layers.Embedding(3, EMBED_DIM, name='bb3_embedding')(input_bb3)
    emb_bb4 = layers.Embedding(3, EMBED_DIM, name='bb4_embedding')(input_bb4)
    emb_bb5 = layers.Embedding(3, EMBED_DIM, name='bb5_embedding')(input_bb5)
    emb_bb6 = layers.Embedding(3, EMBED_DIM, name='bb6_embedding')(input_bb6)
    emb_bb7 = layers.Embedding(3, EMBED_DIM, name='bb7_embedding')(input_bb7)
    emb_bb8 = layers.Embedding(3, EMBED_DIM, name='bb8_embedding')(input_bb8)
    emb_TREND = layers.Embedding(3, EMBED_DIM, name='TREND_embedding')(input_TREND)
    emb_CHANGE = layers.Embedding(4, EMBED_DIM, name='CHANGE_embedding')(input_CHANGE)
    emb_BREAK = layers.Embedding(3, EMBED_DIM, name='BREAK_embedding')(input_BREAK)
    emb_rsi15 = layers.Embedding(4, EMBED_DIM, name='rsi15_embedding')(input_rsi15)
    emb_rsi25 = layers.Embedding(4, EMBED_DIM, name='rsi25_embedding')(input_rsi25)
    emb_rsi35 = layers.Embedding(4, EMBED_DIM, name='rsi35_embedding')(input_rsi35)
    # input features to model 
    features_bb1 = layers.LSTM(128,dropout=0.2)(emb_bb1)
    features_bb2 = layers.LSTM(128,dropout=0.2)(emb_bb2)
    features_bb3 = layers.LSTM(128,dropout=0.2)(emb_bb3)
    features_bb4 = layers.LSTM(128,dropout=0.2)(emb_bb4)
    features_bb5 = layers.LSTM(128,dropout=0.2)(emb_bb5)
    features_bb6 = layers.LSTM(128,dropout=0.2)(emb_bb6)
    features_bb7 = layers.LSTM(128,dropout=0.2)(emb_bb7)
    features_bb8 = layers.LSTM(128,dropout=0.2)(emb_bb8)
    features_TREND = layers.LSTM(128,dropout=0.2)(emb_TREND)
    features_CHANGE = layers.LSTM(128,dropout=0.2)(emb_CHANGE)
    features_BREAK = layers.LSTM(128,dropout=0.2)(emb_BREAK)
    features_rsi15 = layers.LSTM(128,dropout=0.2)(emb_rsi15)
    features_rsi25 = layers.LSTM(128,dropout=0.2)(emb_rsi25)
    features_rsi35 = layers.LSTM(128,dropout=0.2)(emb_rsi35)
    features_wr_atr = layers.LSTM(128,dropout=0.2)(input_wr_atr)
    features_ohlc = layers.LSTM(128, return_sequences=False)(input_ohlc)
    features_all = layers.concatenate([features_bb1, features_bb2, features_bb3, features_bb4, features_bb5, 
                                       features_bb6, features_bb7, features_bb8, features_TREND, features_CHANGE, 
                                       features_BREAK, features_rsi15, features_rsi25, features_rsi35,features_wr_atr,
                                       features_ohlc])
    # dropout, layernormalization 
    
    output_maxp = layers.Dense(1, name='maxp')(features_all)
    output_minp = layers.Dense(1, name='minp')(features_all)
    # bhs labels
    output_buyholdsell = layers.Dense(3, activation="softmax", name = 'bhs')(features_all)
    
    model = keras.Model(
                inputs=[input_ohlc, input_bb1, input_bb2, input_bb3, input_bb4, input_bb5, input_bb6, 
                        input_bb7, input_bb8, input_TREND, input_CHANGE, input_BREAK, input_rsi15, 
                        input_rsi25, input_rsi35,input_wr_atr],
                outputs=[output_maxp, output_minp,output_buyholdsell],
            )

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=0.001),
        loss={
            "maxp": 'mse',
            "minp":'mse',
            'bhs' : 'SparseCategoricalCrossentropy',
        },
        metrics={
            "maxp": tf.keras.metrics.RootMeanSquaredError(),
            "minp": tf.keras.metrics.RootMeanSquaredError(),
            'bhs' :'acc',
        }
    )
    model.summary()
    
    return model 

In [ ]:
LSTM_model = buildModel(20)

In [ ]:
# Test: {batch size : [32, 64 , 128, 256] epoch:[50,100,300,500...]}
history = LSTM_model.fit(train_x_dict, train_y_dict, epochs=30, 
    batch_size=256,verbose = 1, validation_data=(valid_x_dict, valid_y_dict), shuffle = False)

In [ ]:
# save model
LSTM_model.save('AUDCAD/LSTM_model_min10.h5')

In [ ]:
visualize_result(LSTM_model,valid_x_dict, valid_y_dict, val_price_scaler_min, val_price_scaler_max,valid_minprice,valid_maxprice)

visualize_history(history)

## Bi LSTM model 

In [ ]:
def buildModel_Bi_LSTM(seqlen, EMBED_DIM=4): 
    # feature input
    input_ohlc = layers.Input((seqlen,4), dtype=tf.float32, name='OHLC')
    input_bb1 = layers.Input((seqlen,), dtype=tf.int64, name='bb1')
    input_bb2 = layers.Input((seqlen,), dtype=tf.int64, name='bb2')
    input_bb3 = layers.Input((seqlen,), dtype=tf.int64, name='bb3')
    input_bb4 = layers.Input((seqlen,), dtype=tf.int64, name='bb4')
    input_bb5 = layers.Input((seqlen,), dtype=tf.int64, name='bb5')
    input_bb6 = layers.Input((seqlen,), dtype=tf.int64, name='bb6')
    input_bb7 = layers.Input((seqlen,), dtype=tf.int64, name='bb7')
    input_bb8 = layers.Input((seqlen,), dtype=tf.int64, name='bb8')
    input_TREND = layers.Input((seqlen,), dtype=tf.int64, name='TREND')
    input_CHANGE = layers.Input((seqlen,), dtype=tf.int64, name='CHANGE')
    input_BREAK = layers.Input((seqlen,), dtype=tf.int64, name='BREAK')
    input_rsi15 = layers.Input((seqlen,), dtype=tf.int64, name='rsi15')
    input_rsi25 = layers.Input((seqlen,), dtype=tf.int64, name='rsi25')
    input_rsi35 = layers.Input((seqlen,), dtype=tf.int64, name='rsi35')
    input_wr_atr = layers.Input((seqlen,6), dtype=tf.float32, name='wr_atr')
    # categorical embedding layer 
    emb_bb1 = layers.Embedding(3, EMBED_DIM, name='bb1_embedding')(input_bb1)
    emb_bb2 = layers.Embedding(3, EMBED_DIM, name='bb2_embedding')(input_bb2)
    emb_bb3 = layers.Embedding(3, EMBED_DIM, name='bb3_embedding')(input_bb3)
    emb_bb4 = layers.Embedding(3, EMBED_DIM, name='bb4_embedding')(input_bb4)
    emb_bb5 = layers.Embedding(3, EMBED_DIM, name='bb5_embedding')(input_bb5)
    emb_bb6 = layers.Embedding(3, EMBED_DIM, name='bb6_embedding')(input_bb6)
    emb_bb7 = layers.Embedding(3, EMBED_DIM, name='bb7_embedding')(input_bb7)
    emb_bb8 = layers.Embedding(3, EMBED_DIM, name='bb8_embedding')(input_bb8)
    emb_TREND = layers.Embedding(3, EMBED_DIM, name='TREND_embedding')(input_TREND)
    emb_CHANGE = layers.Embedding(4, EMBED_DIM, name='CHANGE_embedding')(input_CHANGE)
    emb_BREAK = layers.Embedding(3, EMBED_DIM, name='BREAK_embedding')(input_BREAK)
    emb_rsi15 = layers.Embedding(4, EMBED_DIM, name='rsi15_embedding')(input_rsi15)
    emb_rsi25 = layers.Embedding(4, EMBED_DIM, name='rsi25_embedding')(input_rsi25)
    emb_rsi35 = layers.Embedding(4, EMBED_DIM, name='rsi35_embedding')(input_rsi35)
    # input feature to model
    features_bb1 = layers.Bidirectional(layers.LSTM(128))(emb_bb1)
    features_bb2 = layers.Bidirectional(layers.LSTM(128))(emb_bb2)
    features_bb3 = layers.Bidirectional(layers.LSTM(128))(emb_bb3)
    features_bb4 = layers.Bidirectional(layers.LSTM(128))(emb_bb4)
    features_bb5 = layers.Bidirectional(layers.LSTM(128))(emb_bb5)
    features_bb6 = layers.Bidirectional(layers.LSTM(128))(emb_bb6)
    features_bb7 = layers.Bidirectional(layers.LSTM(128))(emb_bb7)
    features_bb8 = layers.Bidirectional(layers.LSTM(128))(emb_bb8)
    features_TREND = layers.Bidirectional(layers.LSTM(128))(emb_TREND)
    features_CHANGE = layers.Bidirectional(layers.LSTM(128))(emb_CHANGE)
    features_BREAK = layers.Bidirectional(layers.LSTM(128))(emb_BREAK)
    features_rsi15 = layers.LSTM(128)(emb_rsi15)
    features_rsi25 = layers.LSTM(128)(emb_rsi25)
    features_rsi35 = layers.LSTM(128)(emb_rsi35)
    features_wr_atr = layers.LSTM(128)(input_wr_atr)
    features_ohlc = layers.Bidirectional(layers.LSTM(128))(input_ohlc)
    features_all = layers.concatenate([features_bb1, features_bb2, features_bb3, features_bb4, features_bb5, 
                                       features_bb6, features_bb7, features_bb8, features_TREND, features_CHANGE, 
                                       features_BREAK, features_rsi15, features_rsi25, features_rsi35,
                                       features_wr_atr,features_ohlc])
    # dropout, layernormalization 
    
    
    output_maxp = layers.Dense(1, name='maxp')(features_all)
    output_minp = layers.Dense(1, name='minp')(features_all)
    # bhs label
    output_buyholdsell = layers.Dense(3, activation="softmax", name = 'bhs')(features_all)
   
    
    model = keras.Model(
                inputs=[input_ohlc, input_bb1, input_bb2, input_bb3, input_bb4, input_bb5, input_bb6, 
                       input_bb7, input_bb8, input_TREND, input_CHANGE, input_BREAK,input_rsi15, 
                        input_rsi25, input_rsi35,input_wr_atr ],
                outputs=[output_maxp, output_minp,output_buyholdsell],
            )

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=0.001),
        loss={
            "maxp": 'mse',
            "minp":'mse',
            'bhs' : 'SparseCategoricalCrossentropy',
        },
        metrics={
            "maxp": tf.keras.metrics.RootMeanSquaredError(),
            "minp": tf.keras.metrics.RootMeanSquaredError(),
            'bhs' :'acc',
        }
    )
    model.summary()
    
    return model 

In [ ]:
Bi_LSTM_Model = buildModel_Bi_LSTM(10)

In [ ]:
# Test: {batch size : [32, 64 , 128, 256] epoch:[50,100,300,500...]}
history = Bi_LSTM_Model.fit(train_x_dict, train_y_dict, 
          epochs=30, 
          batch_size=256,verbose = 1, validation_data=(valid_x_dict ,valid_y_dict), shuffle = False)

In [ ]:
# save model
Bi_LSTM_Model.save('path/Bi_LSTM_Model.h5')

In [ ]:
visualize_result(LSTM_model,valid_x_dict, valid_y_dict, val_price_scaler_min, val_price_scaler_max,valid_minprice,valid_maxprice)
# visualize_result(transformer,test_x_dict, test_price_scaler_min,test_price_scaler_max,test_minprice, test_maxprice)
visualize_history(history)

# Transformer

In [ ]:
feat_dim = 256
embed_dim = 64 # Embedding size for attention
head_size = 64
num_heads = 4  # Number of attention heads
ff_dim = 128  # Hidden layer size in feed forward network inside transformer
dropout_rate = 0.3
num_blocks = 2


def get_position_encoding(seq_len, d, n=10000):
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    return P


def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.2):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res
    
def build_model(input_shape,EMBED_DIM, head_size, num_heads, ff_dim, num_transformer_blocks, dropout=0.2, mlp_dropout=0.2):

    # feature input
    input_ohlc = layers.Input((input_shape,4), dtype=tf.float32, name='OHLC')
    input_bb1 = layers.Input((input_shape,), dtype=tf.int64, name='bb1')
    input_bb2 = layers.Input((input_shape,), dtype=tf.int64, name='bb2')
    input_bb3 = layers.Input((input_shape,), dtype=tf.int64, name='bb3')
    input_bb4 = layers.Input((input_shape,), dtype=tf.int64, name='bb4')
    input_bb5 = layers.Input((input_shape,), dtype=tf.int64, name='bb5')
    input_bb6 = layers.Input((input_shape,), dtype=tf.int64, name='bb6')
    input_bb7 = layers.Input((input_shape,), dtype=tf.int64, name='bb7')
    input_bb8 = layers.Input((input_shape,), dtype=tf.int64, name='bb8')
    input_TREND = layers.Input((input_shape,), dtype=tf.int64, name='TREND')
    input_CHANGE = layers.Input((input_shape,), dtype=tf.int64, name='CHANGE')
    input_BREAK = layers.Input((input_shape,), dtype=tf.int64, name='BREAK')
    input_rsi15 = layers.Input((input_shape,), dtype=tf.int64, name='rsi15')
    input_rsi25 = layers.Input((input_shape,), dtype=tf.int64, name='rsi25')
    input_rsi35 = layers.Input((input_shape,), dtype=tf.int64, name='rsi35')
    input_wr_atr = layers.Input((input_shape,6), dtype=tf.float32, name='wr_atr')

    # categorical embedding layer 
    # generate postional embedding
    positional_embedding = get_position_encoding(20,906)
    
    emb_bb1 = layers.Embedding(3, EMBED_DIM, name='bb1_embedding')(input_bb1)
    emb_bb2 = layers.Embedding(3, EMBED_DIM, name='bb2_embedding')(input_bb2)
    emb_bb3 = layers.Embedding(3, EMBED_DIM, name='bb3_embedding')(input_bb3)
    emb_bb4 = layers.Embedding(3, EMBED_DIM, name='bb4_embedding')(input_bb4)
    emb_bb5 = layers.Embedding(3, EMBED_DIM, name='bb5_embedding')(input_bb5)
    emb_bb6 = layers.Embedding(3, EMBED_DIM, name='bb6_embedding')(input_bb6)
    emb_bb7 = layers.Embedding(3, EMBED_DIM, name='bb7_embedding')(input_bb7)
    emb_bb8 = layers.Embedding(3, EMBED_DIM, name='bb8_embedding')(input_bb8)
    emb_TREND = layers.Embedding(3, EMBED_DIM, name='TREND_embedding')(input_TREND)
    emb_CHANGE = layers.Embedding(4, EMBED_DIM, name='CHANGE_embedding')(input_CHANGE)
    emb_BREAK = layers.Embedding(3, EMBED_DIM, name='BREAK_embedding')(input_BREAK)
    emb_rsi15 = layers.Embedding(4, EMBED_DIM, name='rsi15_embedding')(input_rsi15)
    emb_rsi25 = layers.Embedding(4, EMBED_DIM, name='rsi25_embedding')(input_rsi25)
    emb_rsi35 = layers.Embedding(4, EMBED_DIM, name='rsi35_embedding')(input_rsi35)
    
    # INPUT EMBEDDING LAYER
    x = layers.Concatenate()([emb_bb1, emb_bb2, emb_bb3, emb_bb4, emb_bb5, 
                              emb_bb6, emb_bb7, emb_bb8, emb_TREND, emb_CHANGE,
                              emb_BREAK,emb_rsi15,emb_rsi25,emb_rsi35,input_wr_atr, input_ohlc])

    x = x+ positional_embedding
    x = layers.Dense(feat_dim)(x)
    
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    
    # pointwise mlp
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(mlp_dropout)(x)
    output_maxp = layers.Dense(1, name='maxp')(x)
    output_minp = layers.Dense(1, name='minp')(x)
    
    # CLASSIFICATION 
    outputs_bhs = layers.Dense(4, activation="softmax", name='bhs')(x)
    model = keras.Model(
                inputs=[input_ohlc, input_bb1, input_bb2, input_bb3, input_bb4, input_bb5, input_bb6, 
                       input_bb7, input_bb8, input_TREND, input_CHANGE, input_BREAK, input_rsi15,input_rsi25,
                       input_rsi35,input_wr_atr],
                outputs=[output_maxp, output_minp,outputs_bhs],
            )
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        loss={
            "maxp": 'mse',
            "minp":'mse',
            'bhs' : 'SparseCategoricalCrossentropy',
        },
        metrics={
            "maxp": tf.keras.metrics.RootMeanSquaredError(),
            "minp": tf.keras.metrics.RootMeanSquaredError(),
            'bhs' :'acc',
        }
    )
    model.summary()
        
    return model

In [ ]:
transformer = build_model(20, head_size, embed_dim,num_heads, ff_dim, num_blocks, dropout=dropout_rate, mlp_dropout=0.2)

In [ ]:
history = transformer.fit(train_x_dict, train_y_dict, 
                          epochs=30,
                          batch_size=256,verbose = 1, validation_data=(valid_x_dict ,valid_y_dict), shuffle = False)

In [ ]:
# save model
transformer.save('AUDCAD/transformer_min10.h5')

In [ ]:
visualize_result(transformer,valid_x_dict, valid_y_dict, val_price_scaler_min, val_price_scaler_max,valid_minprice,valid_maxprice)

visualize_history(history)
